In [1]:
import json
import os
import time
from pandas import json_normalize
import psycopg2
from psycopg2 import sql

import pandas as pd
import requests
from dotenv import load_dotenv

load_dotenv()

access_token = os.getenv("ACCESS_TOKEN")
HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": "localhost",
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

# Registra o tempo antes da execução
start_prog = time.time()

In [2]:
print(access_token)

APP_USR-6628000663056293-110709-26deb175119a3b99fa12abf1a902638f-233632476


## Verificando dados da conta

In [3]:
url = "https://api.mercadolibre.com/users/me"

payload = {}
headers = {
  'Authorization': f'Bearer {access_token}'
}

response = requests.request("GET", url, headers=headers, data=payload)
response = response.text

print(response)

# Encontre posição do início e fim do campo "id"
start_pos = response.find('"id"')  # Encontra onde começa "id"
end_pos = response.find(',', start_pos)  # Encontra onde termina "id"

# Extraia o valor de "id"
seller_id = response[start_pos:end_pos].split(":")[1]
seller_id = seller_id.strip().strip('"')

print(f'O valor de "seller_id" é: {seller_id}')

{"id":233632476,"nickname":"MUSICALCENTERPG","registration_date":"2016-11-05T17:35:04.000-04:00","first_name":"MUSICAL CENTER PONTA","last_name":"GROSSA EIRELI -","gender":"","country_id":"BR","email":"musicalc168@gmail.com","identification":{"number":"10885874000127","type":"CNPJ"},"address":{"address":"Estrada da Gruta 775","city":"Louveira","state":"BR-SP","zip_code":"13290431"},"phone":{"area_code":"19","extension":"","number":"38787702","verified":false},"alternative_phone":{"area_code":"","extension":"","number":""},"user_type":"brand","tags":["brand","large_seller","eshop","mshops","messages_as_seller"],"logo":null,"points":4513,"site_id":"MLB","permalink":"http://perfil.mercadolivre.com.br/MUSICALCENTERPG","seller_experience":"ADVANCED","bill_data":{"accept_credit_note":"Y"},"seller_reputation":{"level_id":"5_green","power_seller_status":"platinum","transactions":{"canceled":1491,"completed":32430,"period":"historic","ratings":{"negative":0.05,"neutral":0.03,"positive":0.92},"t

## Carregando Inventory Ids de Produtos a serem pesquisados

Esses dados podem ser encontrados n coluna 'Código ML' da planilha 'Relação Full x Tiny' em 'Envios Full.xlsx'

In [4]:
# Especifica caminho para o arquivo Excel
caminho_arquivo_excel = r'../Data/Base/Envios Full.xlsx'

# Especifica nome da planilha
nome_planilha = 'Relação Full x Tiny'

# Especifica nome da coluna que deseja carregar
coluna1 = 'Código ML' 
coluna2 = 'ID do anúncio'
coluna3 = 'ID Tiny'

# Df com colunas específicas
df_ml_estoque_id = pd.read_excel(caminho_arquivo_excel, sheet_name=nome_planilha, usecols=[coluna1,coluna2,coluna3])

# Adiciona prefixo "MLB" a cada linha na coluna especificada
# df_ml_id['ID do anúncio'] = 'MLB' + df_ml_id['ID do anúncio'].astype(str)

# Tamanho do df
df_ml_estoque_id.shape


(885, 3)

In [6]:
# Crie um DataFrame com valores não nulos em 'Código ML'
df_ml_id_no_null = df_ml_estoque_id[~pd.isnull(df_ml_estoque_id['Código ML'])]
df_ml_id_no_null.to_csv('../Data/Output/df_ml_id_no_null.csv', index=False)

# Crie um DataFrame com valores nulos em 'Código ML'
df_ml_id_with_null = df_ml_estoque_id[pd.isnull(df_ml_estoque_id['Código ML'])]
df_ml_id_with_null.to_csv('../Data/Output/df_ml_id_with_null.csv', index=False)

In [7]:
# Verifica se há duplicatas
duplicates = df_ml_id_no_null.duplicated()
n_dup = duplicates.sum()

print(f'Número de duplicatas: {n_dup}')

# Encontre as duplicatas no DataFrame
duplicates = df_ml_id_no_null[df_ml_id_no_null.duplicated(keep=False)]

# Exiba as próprias duplicatas
duplicates

Número de duplicatas: 13


,Código ML,ID do anúncio,ID Tiny
52,KVWJ40778,1233445054,506979980
53,KVWJ40778,1233445054,506979980
252,OFYI79568,1509021649,561968045
253,OFYI79568,1509021649,561968045
269,WXUT79339,1512798100,681951269
270,WXUT79339,1512798100,681951269
272,TDFP80328,1512860650,561967156
273,TDFP80328,1512860650,561967156
285,DUCB06017,1557290709,512617044
286,DUCB06017,1557290709,512617044


In [8]:
# Remove as duplicatas e atualiza o DataFrame
df_ml_id_no_dup = df_ml_id_no_null.drop_duplicates()

df_ml_id_no_dup.shape

(754, 3)

In [9]:
# Qtd de itens sem Inventory ID
df_ml_id_with_null.shape


(118, 3)

In [10]:
df_ml_id_no_dup.head(1)

,Código ML,ID do anúncio,ID Tiny
0,JFGN34621,924922735,509517168


In [11]:
df_ml_ids = df_ml_id_no_dup.copy()

In [12]:
df_ml_ids.columns = ['ml_inventory_id', 'ml_code', 'tiny_id']
df_ml_ids.columns

Index(['ml_inventory_id', 'ml_code', 'tiny_id'], dtype='object')

In [13]:
df_ml_ids.head(1)

,ml_inventory_id,ml_code,tiny_id
0,JFGN34621,924922735,509517168


## Populando tabela tiny_ml_codes no DB

In [14]:

# Ela contém a relação entre tiny_id e inventory_id do ML 
# e não permite pares de valores duplicados entre (ml_inventory_id, tiny_id)

conn = psycopg2.connect(**db_config)

cursor = conn.cursor()

for index, row in df_ml_ids.iterrows():
    insert_query = sql.SQL("INSERT INTO tiny_ml_codes (ml_inventory_id, ml_code, tiny_id) VALUES (%s, %s, %s)")
    cursor.execute(insert_query, (row['ml_inventory_id'], row['ml_code'], row['tiny_id']))

conn.commit()

# Feche o cursor e a conexão
cursor.close()
conn.close()
print('Dados inseridos com sucesso!')

Dados inseridos com sucesso!


## Pegando Estoque de FulFillment

In [53]:
conn = psycopg2.connect(**db_config)

sql_query = "SELECT * FROM tiny_ml_codes"

df_codes = pd.read_sql(sql_query, conn)

df_codes.head(1)


C:\Users\couti\AppData\Local\Temp\ipykernel_18956\1288080312.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_codes = pd.read_sql(sql_query, conn)


,tiny_id,ml_inventory_id,ml_code,created_at,updated_at
0,509517168,JFGN34621,924922735,2023-11-07 15:46:15.284630,None


In [54]:
df_codes.shape

(754, 5)

In [55]:
counter = 0
json_list = []

# df_codes = df_codes.head(25)

for item in df_codes['ml_inventory_id']:
    url = f"https://api.mercadolibre.com/inventories/{item}/stock/fulfillment"

    payload = {}
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    print(f'Buscando dados de: {item}')

    response = requests.get(url, headers=headers, data=payload)
    response_data = response.json()

    json_list.append(response_data)
    
    counter += 1
    
    if counter % 50 == 0:
        print(f"Fazendo uma pausa de 1 minuto...")
        time.sleep(60)

Buscando dados de: JFGN34621
Buscando dados de: ERIM51807
Buscando dados de: HVCS52763
Buscando dados de: ZQIV51979
Buscando dados de: RBUB53192
Buscando dados de: FXMN50419
Buscando dados de: TDFV51283
Buscando dados de: PKNN52962
Buscando dados de: MAEG54107
Buscando dados de: RYWV54124
Buscando dados de: VIKN53907
Buscando dados de: MNSV53911
Buscando dados de: IXGP53982
Buscando dados de: XPLS45081
Buscando dados de: UWGO53377
Buscando dados de: KVWJ40778
Buscando dados de: KBLG86592
Buscando dados de: APFU85474
Buscando dados de: CCQW73335
Buscando dados de: DXJZ75064
Buscando dados de: JDMV76187
Buscando dados de: RJUL81287
Buscando dados de: TNXB83665
Buscando dados de: CIKE94068
Buscando dados de: HGUJ21007
Buscando dados de: ZXFT21237
Buscando dados de: KBOK20943
Buscando dados de: MENA20785
Buscando dados de: CQAQ20361
Buscando dados de: HXJD21466
Buscando dados de: GSNC20904
Buscando dados de: FRDZ21133
Buscando dados de: CNHG19808
Buscando dados de: DJDL21105
Buscando dados

In [57]:
len(json_list)

754

In [58]:
copy_json_list = list(json_list)


df = pd.DataFrame(copy_json_list)

df_er = json_normalize(copy_json_list, record_path='external_references', meta=['inventory_id', 'total', 'available_quantity', 'not_available_quantity', 'not_available_detail'])
df_nad = json_normalize(copy_json_list, record_path='not_available_detail', meta=['inventory_id', 'total', 'available_quantity', 'not_available_quantity', 'external_references'])


print('df',df.shape)
print('df_er',df_er.shape)
print('df_nad',df_nad.shape)

df_er.head(3)

df (754, 6)
df_er (754, 8)
df_nad (81, 7)


,type,id,variation_id,inventory_id,total,available_quantity,not_available_quantity,not_available_detail
0,item,MLB924922735,NaN,JFGN34621,71,71,0,[]
1,item,MLB949788598,NaN,ERIM51807,36,33,3,"[{'status': 'transfer', 'quantity': 3}]"
2,item,MLB950214971,NaN,HVCS52763,3,1,2,"[{'status': 'transfer', 'quantity': 2}]"


In [59]:
df_nad.head(3)

,status,quantity,inventory_id,total,available_quantity,not_available_quantity,external_references
0,transfer,3,ERIM51807,36,33,3,"[{'type': 'item', 'id': 'MLB949788598'}]"
1,transfer,2,HVCS52763,3,1,2,"[{'type': 'item', 'id': 'MLB950214971'}]"
2,transfer,3,RBUB53192,21,18,3,"[{'type': 'item', 'id': 'MLB950295955'}]"


In [60]:
x = df_nad[df_nad['inventory_id'] == 'ERIM51807']
y = df_er[df_er['inventory_id'] == 'ERIM51807']

x

,status,quantity,inventory_id,total,available_quantity,not_available_quantity,external_references
0,transfer,3,ERIM51807,36,33,3,"[{'type': 'item', 'id': 'MLB949788598'}]"


In [61]:
y

,type,id,variation_id,inventory_id,total,available_quantity,not_available_quantity,not_available_detail
1,item,MLB949788598,NaN,ERIM51807,36,33,3,"[{'status': 'transfer', 'quantity': 3}]"


In [62]:
print(df_nad.columns)
print(df_er.columns)

Index(['status', 'quantity', 'inventory_id', 'total', 'available_quantity',
       'not_available_quantity', 'external_references'],
      dtype='object')
Index(['type', 'id', 'variation_id', 'inventory_id', 'total',
       'available_quantity', 'not_available_quantity', 'not_available_detail'],
      dtype='object')


In [63]:
df_nad_ = df_nad.drop(columns='external_references')
df_er_ = df_er.drop(columns='not_available_detail')

In [64]:
common_cols = ['inventory_id', 'total', 'available_quantity', 'not_available_quantity']
# df_fulfillment = df_er_.merge(df_nad_, on=common_cols, how='left', suffixes=('_er','_nad'))
df_fulfillment = df_er_.merge(df_nad_, on=common_cols, how='left')

df_fulfillment.shape

(821, 9)

In [65]:
df_fulfillment.head(1)

,type,id,variation_id,inventory_id,total,available_quantity,not_available_quantity,status,quantity
0,item,MLB924922735,NaN,JFGN34621,71,71,0,NaN,NaN


In [66]:
map_cols = {'inventory_id': 'ml_inventory_id', 'id': 'ml_item_id', 'status': 'nad_status','quantity':'nad_quantity'}
df_fulfillment = df_fulfillment.rename(columns=map_cols)
order_col = ['ml_inventory_id','ml_item_id','variation_id','nad_status','nad_quantity','total','available_quantity','not_available_quantity','type']
df_fulfillment = df_fulfillment[order_col]

df_fulfillment.head(1)

,ml_inventory_id,ml_item_id,variation_id,nad_status,nad_quantity,total,available_quantity,not_available_quantity,type
0,JFGN34621,MLB924922735,NaN,NaN,NaN,71,71,0,item


In [67]:
df_fulfillment.shape

(821, 9)

In [68]:
df_fulfillment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 821 entries, 0 to 820
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ml_inventory_id         821 non-null    object 
 1   ml_item_id              821 non-null    object 
 2   variation_id            313 non-null    float64
 3   nad_status              143 non-null    object 
 4   nad_quantity            143 non-null    float64
 5   total                   821 non-null    object 
 6   available_quantity      821 non-null    object 
 7   not_available_quantity  821 non-null    object 
 8   type                    821 non-null    object 
dtypes: float64(2), object(7)
memory usage: 57.9+ KB


In [69]:
df_fulfillment['variation_id'] = df_fulfillment['variation_id'].astype(str)
df_fulfillment['nad_quantity'] = df_fulfillment['nad_quantity'].fillna(0).astype(int)

In [70]:
df_fulfillment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 821 entries, 0 to 820
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ml_inventory_id         821 non-null    object
 1   ml_item_id              821 non-null    object
 2   variation_id            821 non-null    object
 3   nad_status              143 non-null    object
 4   nad_quantity            821 non-null    int32 
 5   total                   821 non-null    object
 6   available_quantity      821 non-null    object
 7   not_available_quantity  821 non-null    object
 8   type                    821 non-null    object
dtypes: int32(1), object(8)
memory usage: 54.6+ KB


In [71]:
import psycopg2
from psycopg2 import sql

# Suponha que você já tenha seu DataFrame df_fulfillment

# Conecte-se ao banco de dados
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

# Itere pelas linhas do DataFrame e insira os dados na tabela
for index, row in df_fulfillment.iterrows():
    insert_query = sql.SQL("""
        INSERT INTO ml_fulfillment3 (ml_inventory_id, ml_item_id, variation_id, nad_status, nad_quantity, total, available_quantity, not_available_quantity, type)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """)
    cursor.execute(insert_query, (row['ml_inventory_id'], row['ml_item_id'], row['variation_id'], row['nad_status'], row['nad_quantity'], row['total'], row['available_quantity'], row['not_available_quantity'], row['type']))

# Confirme as alterações
conn.commit()

# Feche o cursor e a conexão
cursor.close()
conn.close()

print('Dados inseridos com sucesso!')


Dados inseridos com sucesso!
